In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
model = load_model('model.keras') 

In [13]:
class_labels = ["glass", "paper", "cardboard", "plastic", "metal", "trash" ]

In [22]:
# Kamerayı aç
camera = cv2.VideoCapture(0)  # Web kamerası için 0 kullanılır

print("Kameradan gerçek zamanlı görüntü akışı başlatıldı.")

detected_class = None  # Tespit edilen sınıfı saklamak için değişken
min_confidence = 0.95  # Gerekli güven seviyesi

while True:
    # Kameradan bir kare al
    ret, frame = camera.read()
    if not ret:
        print("Kamera görüntüsü alınamadı.")
        break

    # Görüntüyü modelin kabul edeceği boyuta ölçekleyin
    input_image = cv2.resize(frame, (224, 224))  # Modelin giriş boyutuna göre ayarlayın
    input_image = np.expand_dims(input_image, axis=0)  # Batch boyutu ekleyin
    input_image = input_image / 255.0  # Normalize edin

    # Modeli kullanarak tahmin yap
    predictions = model.predict(input_image)
    confidence = np.max(predictions[0])
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_labels[predicted_class_index]

    # Tespit edilen sınıfı ve güven seviyesini kontrol et
    if confidence > min_confidence:
        detected_class = predicted_class
        print(f"Model {confidence * 100:.2f}% emin. Tespit edilen sınıf: {predicted_class}")
        break  # Güven seviyesi yeterli olduğunda döngüden çık

    # Tespit edilen nesneyi çerçeveye alın
    # Nesne konumunu bulmak için görüntünün gri tonlamasını alın
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY)

    # Kenarları tespit edin
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # En büyük konturu bulun
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Dikdörtgeni çizin
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Sınıfı ve güven seviyesini kutunun üstüne yazın
        label = f"{predicted_class} ({confidence * 100:.2f}%)"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Görüntüyü göster
    cv2.imshow("Real-Time Object Detection", frame)

    # 'q' tuşuna basarak manuel olarak çıkabilirsiniz
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Kapatma komutu alındı. Kamerayı kapatıyorum.")
        break

# Kamerayı kapat ve kaynakları serbest bırak
camera.release()
cv2.destroyAllWindows()

# Tespit edilen sınıfı döndür
if detected_class:
    print(f"Son tespit edilen sınıf: {detected_class}")
else:
    print("Herhangi bir sınıf yeterli güven düzeyiyle tespit edilemedi.")

Kameradan gerçek zamanlı görüntü akışı başlatıldı.
1/1 [==============================] - 0s 62ms/step
Model 95.74% emin. Tespit edilen sınıf: trash
Son tespit edilen sınıf: trash
